In [37]:
from app.main.models import EducationStaff
import requests
from config import ApeksAPI
from app.main.func import db_filter_req, db_request
from datetime import date
from calendar import monthrange


def get_lessons(year, month):
    """List of lessons for selected month"""
    first_day = 1
    last_day = monthrange(year, month)[1]

    params = {
        "token": ApeksAPI.TOKEN,
        "table": 'schedule_day_schedule_lessons',
        "filter": "date between '" + date(year, month, first_day).isoformat() + "' and '" + date(year, month, last_day).isoformat() + "'"
    }
    resp = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params)
    return resp.json()['data']


def get_departments():
    """Getting list of department as dict id:[name, short_name]"""
    dept_list = {}
    resp = db_filter_req('state_departments', 'parent_id', ApeksAPI.APEKS_DEPT_ID)
    for dept in resp:
        dept_list[dept['id']] = [dept['name'], dept['name_short']]
    return dept_list


def lessons_staff():
    """Get staff for lesson_id ( {lesson_id:staff_id} )"""
    lessons_staff = {}
    resp = db_request('schedule_day_schedule_lessons_staff')
    for less in resp:
        if less.get('lesson_id') in lessons_staff:
            lessons_staff[less.get('lesson_id')].append(less.get('staff_id'))
        else:
            lessons_staff[less.get('lesson_id')] = [less.get('staff_id')]
    return lessons_staff


def load_subgroups():
    """Get group_id for subgroup_id ( {subgroup_id:group_id} )"""
    subgroups = {}
    resp = db_request('load_subgroups')
    for sub in resp:
        subgroups[sub.get('id')] = sub.get('group_id')
    return subgroups


def load_groups():
    """Get group info by group_id ( {id:[{}] )"""
    groups = {}
    resp = db_request('load_groups')
    for group in resp:
        groups[group.get('id')] = group
    return groups


def plan_education_plans_education_forms():
    """Get education_form_id info by education_plan_id ( {education_plan_id:education_form_id} )"""
    education_forms = {}
    resp = db_request('plan_education_plans_education_forms')
    for plan in resp:
        education_forms[plan.get('education_plan_id')] = plan.get('education_form_id')
    return education_forms


def plan_education_plans():
    """Get plan info by plan_id ( {id:[{}] )"""
    education_plans = {}
    resp = db_request('plan_education_plans')
    for plan in resp:
        education_plans[plan.get('id')] = plan
    return education_plans


class LoadReport:
    def __init__(self, year, month):
        self.year = year
        self.month = month
        self.departments = get_departments()
        self.prepod_dept_structure = self.prepod_dept_structure()
        self.lesson_staff = lessons_staff()
        self.load_subgroups = load_subgroups()
        self.load_groups = load_groups()
        self.plan_education_plans_education_forms = plan_education_plans_education_forms()
        self.plan_education_plans = plan_education_plans()
        self.structured_lessons = self.structured_lessons()

    def prepod_dept_structure(self):
        """Department Id for prepod in current month ( {staff_id:department_id} )"""
        structure = {}
        staff = EducationStaff(self.year, self.month)
        for d in self.departments:
            staff_list = staff.staff_list(d)
            for s in staff_list:
                structure[s] = d
        return structure

    def structured_lessons(self):
        """Current month lessons list with all data for load report"""
        structured_lessons = []
        for lesson in get_lessons(self.year, self.month):
            if not lesson.get('group_id'):
                if lesson.get('subgroup_id'):
                    lesson['group_id'] = self.load_subgroups.get(lesson.get('subgroup_id'))
            education_plan_id = self.load_groups.get(lesson.get('group_id'))['education_plan_id']
            education_form_id = self.plan_education_plans_education_forms.get(education_plan_id)
            education_level_id = self.plan_education_plans.get(education_plan_id)['education_level_id']
            lesson['education_plan_id'] = education_plan_id
            lesson['education_form_id'] = education_form_id
            lesson['education_level_id'] = education_level_id
            if lesson.get('id') in self.lesson_staff:
                for staff in self.lesson_staff.get(lesson.get('id')):
                    lesson['staff_id'] = staff
                    lesson['department_id'] = self.prepod_dept_structure.get(staff)
                    structured_lessons.append(lesson)
        return structured_lessons

    def unknown_lessons(self):
        """Lessons with inactive staff (which doesn't work in department in selected time)"""
        unknown = []
        for l in self.structured_lessons:
            if not l.get('department_id'):
                unknown.append(l)
        return unknown

    def department_lessons(self, department_id):
        """Select department lessons for load report"""
        dept_lessons = []
        for l in self.structured_lessons:
            if l.get('department_id') == str(department_id):
                dept_lessons.append(l)
        return dept_lessons

In [45]:
load = LoadReport(2021, 12)

In [46]:
load.department_lessons(12)

[{'id': '19420',
  'schedule_id': '139',
  'discipline_id': '71',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '5',
  'topic_name': 'Анализ, обобщение, интерпретация и представление статистических данных',
  'classroom_id': '3',
  'group_id': '32',
  'subgroup_id': '49',
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '13917',
  'education_plan_id': '65',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '33',
  'department_id': '12'},
 {'id': '19421',
  'schedule_id': '139',
  'discipline_id': '71',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '5',
  'topic_name': 'Анализ, обобщение, интерпретация и представление статистических данных',
  'classroom_id': '69',
  'group_id': '32',
  'subgroup_id': '50',
  'flow': None,
  'is_empty': '0',
  'self_work